In [1]:
import pandas as pd
import datetime
import numpy as np
import

In [2]:
ACTIVITY_DATA = "/Users/neeraj/Documents/Hackathons/data_brown_datathon/TripAdvisor/activity_data.csv"
HOTEL_DATA = "/Users/neeraj/Documents/Hackathons/data_brown_datathon/TripAdvisor/hotel_data.csv"

In [3]:
if not os.path.exists("tmp_network.csv"):
    activity_df = pd.read_csv(ACTIVITY_DATA)
    hotel_df = pd.read_csv(HOTEL_DATA).set_index('hotel_id')
    activity_df.head()

    to_timestamp = lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').timestamp()
    activity_df['timestamp'] = activity_df['date'].apply(to_timestamp)
    #PUT IN GEOLOCATOR LATER FOR HOTELS AND FOR USERS

    def encode(a):
        encodings = {'view':1, 'price_click':2, 'hotel_website_click':3, 'booking':4}
        return encodings[a]

    activity_df['user_action'] = activity_df['user_action'].apply(encode)
    activity_df = activity_df.drop(columns = ['date','user_country','device'])

    hotel_df = hotel_df['hotel_name']

    df = activity_df.join(hotel_df, how = 'inner', on = 'hotel_id')
else:
    df = pd.read_csv("tmp_network.csv")    
df.head()

,user_id,hotel_id,user_action,timestamp,hotel_name
0,314066,570101,1,1.546837e+09,La Quinta Inn Queens (New York City)
974,229655,570101,1,1.547442e+09,La Quinta Inn Queens (New York City)
2409,135830,570101,1,1.547960e+09,La Quinta Inn Queens (New York City)
2840,222524,570101,2,1.546924e+09,La Quinta Inn Queens (New York City)
4468,51835,570101,1,1.547096e+09,La Quinta Inn Queens (New York City)


In [4]:
df.to_csv("tmp_network.csv")
import networkx as nx

In [5]:
dft = df.iloc[:1000, :]
user_ids = [-1*v for v in list(np.unique(dft.user_id))]
hotel_ids = list(np.unique(dft.hotel_id))

In [6]:
g = nx.Graph()
g.add_nodes_from(user_ids)
g.add_nodes_from(hotel_ids)
E = []
for user_id, hotel_id in zip(dft.user_id, dft.hotel_id):
    E.append((-1*user_id, hotel_id))
g.add_edges_from(E)

In [7]:
pos=nx.fruchterman_reingold_layout(g) 

In [8]:
Xv=[pos[k][0] for k in user_ids] + [pos[k][0] for k in hotel_ids]
Yv=[pos[k][1] for k in user_ids] + [pos[k][1] for k in hotel_ids]
Xed=[]
Yed=[]

for edge in E:
    Xed+=[pos[edge[0]][0],pos[edge[1]][0], None]
    Yed+=[pos[edge[0]][1],pos[edge[1]][1], None] 

In [10]:
import plotly.plotly as py
from plotly.graph_objs import *

In [ ]:
width=800
height=800
axis=dict(showline=False, # hide axis line, grid, ticklabels and  title
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title='' 
          )
layout=Layout(title= "Coauthorship network of scientists working on network theory and experiment"+\
              "<br> Data source: <a href='https://networkdata.ics.uci.edu/data.php?id=11'> [1]</a>",  
    font= dict(size=12),
    showlegend=False,
    autosize=False,
    width=width,
    height=height,
    xaxis=layout.XAxis(axis),
    yaxis=layout.YAxis(axis),          
    margin=layout.Margin(
        l=40,
        r=40,
        b=85,
        t=100,
    ),
    hovermode='closest',
    annotations=[
           dict(
           showarrow=False, 
            text='This igraph.Graph has the Kamada-Kawai layout',  
            xref='paper',     
            yref='paper',     
            x=0,  
            y=-0.1,  
            xanchor='left',   
            yanchor='bottom',  
            font=dict(
            size=14 
            )     
            )
        ]          
    )
trace3=Scatter(x=Xed,
               y=Yed,
               mode='lines',
               line=dict(color='rgb(210,210,210)', width=1),
               hoverinfo='none'
               )
trace4=Scatter(x=Xv,
               y=Yv,
               mode='markers',
               name='net',
               marker=dict(symbol='circle-dot',
                             size=5, 
                             color='#6959CD',
                             line=dict(color='rgb(50,50,50)', width=0.5)
                             ),
               hoverinfo='text'
               )

annot="This networkx.Graph has the Fruchterman-Reingold layout<br>Code:"+\
"<a href='http://nbviewer.ipython.org/gist/empet/07ea33b2e4e0b84193bd'> [2]</a>"

data1=[trace3, trace4]
fig1=Figure(data=data1, layout=layout)
fig1['layout']['annotations'][0]['text']=annot
py.iplot(fig1, filename='Coautorship-network-nx')